In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn import ensemble
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO 


from IPython.display import Image 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [44]:
df = pd.read_csv('data/train_data.csv')

In [48]:
x_train, x_test, y_train, y_test = train_test_split(
    df[[
        'bedrooms',
        'bathrooms',
        'sqft_living',
        'sqft_lot',
        'floors',
        'waterfront',
        'view',
        'condition',
        'grade',
        'sqft_above',
        'sqft_basement',
        'yr_built',
        'yr_renovated',
        'sqft_living15',
        'sqft_lot15',
        'zipcode',
        'lat',
        'long'
    ]], 
    df['price'], 
    random_state=1
)

In [52]:
params = {
    'n_estimators': 200,
    'max_depth': 5,
    'learning_rate': 0.1,
    'criterion': 'mse'
}

# gradient_boosting_regressor = GridSearchCV(
#     ensemble.GradientBoostingRegressor(),
#     params,
# )

gradient_boosting_regressor = ensemble.GradientBoostingRegressor(**params)
gradient_boosting_regressor.fit(x_train, y_train)
# print(gradient_boosting_regressor.best_params_)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=5,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=200,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [53]:
def rmse(predictions, actual):
    return np.sqrt(np.mean((predictions - actual)**2))

print('Gradient Boost', rmse(gradient_boosting_regressor.predict(x_test), y_test))
print(gradient_boosting_regressor.score(x_test, y_test))

Gradient Boost 121404.11284512942
0.8915384177487046


# Submission File Generation

In [ ]:
dfTest = pd.read_csv('data/val_data.csv').rename({'Unnamed: 0': 'Unique_idx'}, axis='columns')

In [ ]:
dfTest['price'] = regr.predict(dfTest[[
    'bedrooms',
    'bathrooms',
    'sqft_living',
    'sqft_lot',
    'floors',
    'waterfront',
    'view',
    'condition',
    'grade',
    'sqft_above',
    'sqft_basement',
    'yr_built',
    'yr_renovated',
    'sqft_living15',
    'sqft_lot15',
]])

In [ ]:
dfTest[['Unique_idx', 'price']].to_csv('submission.csv', index=None, header=True)